In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

import intake

import config
import util

In [23]:
cat = intake.open_catalog("catalogs/fluxes-catalog-web.yml")
print(cat.yaml())

sources:
  fluxes-catalog-web:
    args:
      path: catalogs/fluxes-catalog-web.yml
    description: Flux datasets
    driver: intake.catalog.local.YAMLFileCatalog
    metadata: {}



In [16]:
ds = cat.fgco2_MPI_SOM_FFN.to_dask().compute()            
ds

<xarray.Dataset>
Dimensions:         (bnds: 2, lat: 180, lon: 360, time: 432)
Coordinates:
  * time            (time) datetime64[ns] 1982-01-15 1982-02-15 ... 2017-12-15
  * lat             (lat) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon             (lon) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
Dimensions without coordinates: bnds
Data variables: (12/14)
    spco2_raw       (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    fgco2_raw       (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    spco2_smoothed  (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    fgco2_smoothed  (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    sol             (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    kw              (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    ...              ...
    dco2_raw        (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    dco2_smoothed   (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    seamask         (lat, lon) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    time_bnds       (time, bnds) datetime64[ns] 1981-12-31 ... 2017-12-31
    lat_bnds        (lat, bnds) float32 -90.0 -89.0 -89.0 ... 89.0 89.0 90.0
    lon_bnds        (lon, bnds) float32 -180.0 -179.0 -179.0 ... 179.0 180.0
Attributes:
    institution:    MPI-MET, Hamburg, Germany (former: ETH Zurich, Switzerland)
    institude_id:   MPI
    model_id:       SOM-FFN
    run_id:         v2018
    contact:        Peter Landschutzer (peter.landschuetzer@mpimet.mpg.de)
    creation_date:  2019-03-21

In [17]:
ds['area'] = util.compute_grid_area(ds)

In [24]:
ds = cat.fgapo_CarbonScope_apo99_v2020.to_dask().compute()            
ds

FSTimeoutError: 

In [3]:
cluster, client = util.get_ClusterClient()
cluster.adapt(minimum_jobs=2, maximum_jobs=5)
cluster

/glade/work/mclong/miniconda3/envs/sno/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44125 instead
  f"Port {expected} is already in use.\n"


In [7]:
%%time
def retrieve_dataset(value):
    try:
        ds = cat.fgco2_fff_odiac(year=value.year, month=value.month, day=value.day).to_dask()
    except Exception:
        ds = None
        raise
    return ds


date_range = pd.date_range(start='2000-01-01', end='2019-01-01')
ds_list = client.map(retrieve_dataset, date_range)
ds_list = client.gather(ds_list)
#ds_list = [ds for ds in ds_list if ds is not None]
#ds = xr.concat(ds_list, dim='time', compat='override', coords='minimal').squeeze()
ds_list

CPU times: user 24 s, sys: 871 ms, total: 24.8 s
Wall time: 1min 13s


[<xarray.Dataset>
 Dimensions:                (n_hour: 24, n_lat: 180, n_lon: 360, n_itime: 6)
 Dimensions without coordinates: n_hour, n_lat, n_lon, n_itime
 Data variables:
     emission               (n_hour, n_lat, n_lon) float64 dask.array<chunksize=(24, 180, 360), meta=np.ndarray>
     surface_area           (n_lat, n_lon) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
     hourly_total_emission  (n_hour) float64 dask.array<chunksize=(24,), meta=np.ndarray>
     itime                  (n_hour, n_itime) int16 dask.array<chunksize=(24, 6), meta=np.ndarray>
 Attributes:
     total_emis:         18.27 TgC
     odiac_total:  Original ODIAC emission for this month =       18.94 TgC/day,
 <xarray.Dataset>
 Dimensions:                (n_hour: 24, n_lat: 180, n_lon: 360, n_itime: 6)
 Dimensions without coordinates: n_hour, n_lat, n_lon, n_itime
 Data variables:
     emission               (n_hour, n_lat, n_lon) float64 dask.array<chunksize=(24, 180, 360), meta=np.ndarray>
     

NameError: name 'client' is not defined